In [5]:
import pandas as pd
import numpy as np
from keras.models import load_model
from datetime import datetime

# 데이터 로드 및 전처리
file_path1 = '/home/ubuntu/project/4.5HZ/Data/완성/플레이리스트/dummy_user(12월26일).csv'
file_path2 = '/home/ubuntu/project/4.5HZ/Data/완성/플레이리스트/playlist_중복제거(12월26일).csv'
file_path3 = '/home/ubuntu/project/4.5HZ/Data/완성/플레이리스트/플레이리스트_협업필터링(12월26일).csv'

df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)

# 필요한 데이터프레임을 병합하고 전처리
df = pd.merge(df2, df3, left_on='playlist_id', right_on='PLAYLIST_ID')
df = pd.merge(df, df1, left_on='USER_ID', right_on='id')

# 필요한 컬럼만 선택
df = df[['USER_ID', 'TRACK_ID', 'rating', 'birth_date', 'gender']]

# 데이터 전처리
# 나이대 계산
df['birth_date'] = pd.to_datetime(df['birth_date'])
df['age'] = (datetime.now().year - df['birth_date'].dt.year) // 10

# 사용자와 아이템의 ID를 숫자로 변환
user_ids = df['USER_ID'].unique()
track_ids = df['TRACK_ID'].unique()

num_users = len(user_ids)
num_tracks = len(track_ids)

user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
track2track_encoded = {x: i for i, x in enumerate(track_ids)}
track_encoded2track = {i: x for i, x in enumerate(track_ids)}

df['USER'] = df['USER_ID'].map(user2user_encoded)
df['TRACK'] = df['TRACK_ID'].map(track2track_encoded)

# 컬럼 타입을 category로 변환
df['age'] = df['age'].astype('category').cat.codes
df['gender'] = df['gender'].astype('category').cat.codes

# 모델 로드
model = load_model('/home/ubuntu/project/4.5HZ/추천시스템/협업필터링/Collaborative_Filtering_model_2.h5')

# 사용자 정보와 추천 생성 함수 정의
def get_user_info(user_id):
    # 사용자 정보를 불러옵니다.
    user_info = df1.loc[df1['id'] == user_id]

    if user_info.empty:
        # 사용자 ID가 데이터프레임에 없을 경우 처리
        print(f"ID가 {user_id}인 사용자를 찾을 수 없습니다.")
        return None, None, None, None

    # 사용자의 나이대와 성별을 반환합니다.
    birth_year = datetime.strptime(user_info['birth_date'].values[0], "%Y-%m-%d").year
    age = (datetime.now().year - birth_year) // 10
    gender = 1 if user_info['gender'].values[0] == 'M' else 0

    # 사용자의 닉네임과 사용자명을 반환합니다.
    nickname = user_info['nickname'].values[0]
    username = user_info['username'].values[0]
    birth_date = user_info['birth_date'].values[0]

    return age, gender, nickname, username, birth_date

# 사용자 ID를 입력으로 받습니다.
user_id = 1

# get_user_info 함수를 사용하여 사용자 정보를 불러옵니다.
age, gender, nickname, username, birth_date = get_user_info(user_id)

# 정보 출력
if age is not None:  # None이 아닌 경우에만 정보를 출력합니다.
    #print(f"사용자 {user_id}의 닉네임: {nickname}")
    print(f"사용자 {user_id}의 사용자명: {username}")
    print(f"사용자 {user_id}의 생년월일: {birth_date}")
    print(f"사용자 {user_id}의 나이: {age}0대")
    print(f"사용자 {user_id}의 성별: {'남성' if gender == 1 else '여성'}")

    # 모든 음악에 대해 해당 사용자의 평점을 예측합니다.
    user_array = np.array([user_id for _ in range(num_tracks)])
    age_array = np.array([age for _ in range(num_tracks)])
    gender_array = np.array([gender for _ in range(num_tracks)])
    track_array = np.array(range(num_tracks))

    inputs = [user_array, track_array, age_array, gender_array]
    predictions = model.predict(inputs)

    # 가장 높은 점수를 받은 음악 5개를 추천합니다.
    top_track_indices = np.argsort(predictions[:, 0])[::-1]
    recommended_track_ids = [track_encoded2track[i] for i in top_track_indices[:5]]
    recommended_track_scores = [predictions[i][0] for i in top_track_indices[:5]]
    
    #print(f"사용자 {user_id}를 위한 추천 음악 ID: {recommended_track_ids}")
    
 # 추천 음악 ID와 예상 평점을 함께 출력
    print(f"\n사용자님을 위한 음악 추천 ")
    #print(f"\nUser ID: {username}님에게 추천하는 음악_2")
    for track_id_2, score in zip(recommended_track_ids, recommended_track_scores):
        print(f"Track ID: {track_id_2}, 예상 평점: {score}")

사용자 1의 사용자명: 정유진
사용자 1의 생년월일: 1999-12-29
사용자 1의 나이: 20대
사용자 1의 성별: 여성
1778/1778 [==============================] - 2s 1ms/step

사용자님을 위한 음악 추천 
Track ID: 4L8pKwvlbzvw8HKwgQM4Mz, 예상 평점: 2.542133331298828
Track ID: 62D6Bjz4IgTKDJRLPD3gJY, 예상 평점: 2.529088258743286
Track ID: 1ODqKQQkoP8KyXSpWASI60, 예상 평점: 2.5073540210723877
Track ID: 1zKEyeeSWHOyd8csMC1WTO, 예상 평점: 2.5073540210723877
Track ID: 7LJZyNjN5U0LdYSBQ9hlLz, 예상 평점: 2.5073540210723877
